In [1]:
import sqlparse

In [2]:
query = """
SELECT
  deviceCategory,
  date,
  brand,
  COUNT(DISTINCT IF(path LIKE '/home.page%'
      OR screenName LIKE 'Home Screen',
      hit_string,
      NULL)) AS home_pageviews,
  COUNT(DISTINCT IF(path LIKE '%.prd%',
      hit_string,
      NULL)) AS product_pageviews,
  COUNT(DISTINCT IF(path LIKE '%.end%',
      hit_string,
      NULL)) AS gallery_pageviews,
  COUNT(DISTINCT IF(path LIKE '%/myaccount/%',
      hit_string,
      NULL)) AS myaccount_pageviews,
  COUNT(DISTINCT IF(path LIKE '/basket.page%',
      hit_string,
      NULL)) AS basket_pageviews,
  COUNT(DISTINCT IF(path LIKE '%/checkout/%',
      hit_string,
      NULL)) AS checkout_pageviews,
  COUNT(DISTINCT hit_string) pageviews_total
FROM (
  SELECT
    date,
    hit_string,
    path,
    screenName,
    CASE
      WHEN source_property IN ('Very App',  'www.Very.co.uk',  'www.very.co.uk') THEN 'Very'
      WHEN source_property IN ('Littlewoods App',
      'www.littlewoods.com') THEN 'Littlewoods'
      WHEN source_property IN ('www.veryexclusive.com' ) THEN 'Very Exclusive'
      WHEN source_property IN ('www.littlewoodsireland.ie') THEN 'Littlewoods Ireland'
      ELSE '?'
    END AS brand,
    CASE
      WHEN hitType = 'APPVIEW' THEN 'App'
      ELSE deviceCategory
    END AS deviceCategory
  FROM (
    SELECT
      CONCAT(fullVisitorId, CAST(visitNumber AS string), CAST(h.hitNumber AS string)) AS hit_string,
      IF(h.sourcePropertyInfo.sourcePropertyDisplayName IS NULL,
        h.page.hostname,
        h.sourcePropertyInfo.sourcePropertyDisplayName) AS source_property,
      device.deviceCategory AS deviceCategory,
      h.page.pagePath AS path,
      h.appInfo.screenName AS screenName,
      h.type AS hitType,
      date
    FROM
      `pure-chariot-776.93481122.ga_sessions_*`,
      UNNEST(hits) h
    WHERE
      _TABLE_SUFFIX BETWEEN FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 5000 DAY))
      AND FORMAT_DATE("%Y%m%d", DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY))
      AND h.type IN('PAGE',
        'APPVIEW')
      AND (h.sourcePropertyInfo.sourcePropertyDisplayName IN( 'www.Very.co.uk',
          'www.littlewoods.com',
          'www.veryexclusive.com',
          'www.littlewoodsireland.ie',
          'Very App',
          'Littlewoods App' )
        OR h.page.hostname IN('www.very.co.uk',
          'www.littlewoods.com',
          'www.veryexclusive.com',
          'www.littlewoodsireland.ie' ))))
GROUP BY
  date,
  deviceCategory,
  brand
"""

In [3]:
var = sqlparse.parse(query)